In [159]:
import csv
import sys
import pandas as pd
import string
import math

TRAIN_SIZE = 80.0

# get user input string
# user_input = input("Enter a review: ")

# initialize TRAIN_SIZE (from user)
input_train_size = int(input("Enter the percentage of data to train on (20-80): "))

#input_train_size = sys.argv[1]
if input_train_size < 20 or input_train_size > 80:
    TRAIN_SIZE = 80
else:
   TRAIN_SIZE = input_train_size




data_pre = pd.read_csv("bumble_google_play_reviews.csv")
data = data_pre.dropna(subset=['content']).reset_index(drop=True)
data['review_class'] = data['score'].apply(lambda x: 'positive' if x > 2 else 'negative')


# 80-20 split
test_index = int(len(data) * 0.8)
train_index = int(len(data) * float(TRAIN_SIZE / 100.0))
train_data = data.iloc[:train_index]
test_data = data.iloc[test_index:]

pos_indices = []
neg_indices = []

for x in range(len(train_data)):
    if train_data['review_class'][x] == 'positive':
        pos_indices.append(x)
    if train_data['review_class'][x] == 'negative':
        neg_indices.append(x)


prior_probabilities = {
    'positive': len(train_data[train_data['review_class'] == 'positive']) / len(train_data),
    'negative': len(train_data[train_data['review_class'] == 'negative']) / len(train_data)
}


In [160]:
vocabulary = []
translation_table = str.maketrans('', '', string.punctuation)

for x in range(len(data['content'])):
    content = data['content'][x]
    lowercase_content = content.lower()
    content_no_punct = lowercase_content.translate(translation_table)
    vocabulary.extend(content_no_punct.split())

unique_vocabulary = []
for item in vocabulary:
    if item not in unique_vocabulary:
        unique_vocabulary.append(item)

In [161]:
binary_bag = [[0]*len(unique_vocabulary) for _ in range(len(train_data))]

for x in range(len(train_data)):
    for y in range(len(train_data['content'][x].split())):
        words_in_one_sentence = (train_data['content'][x].split()[y]).lower().translate(translation_table)
        if words_in_one_sentence != '':   
            index_of_word = unique_vocabulary.index(words_in_one_sentence)
            binary_bag[x][index_of_word] = 1

# sigma(z) = 1 / (1 + e^(-z))

num_docs_including_word = [0]*len(unique_vocabulary)
num_pos_docs_including_word = [0]*len(unique_vocabulary)
num_neg_docs_including_word = [0]*len(unique_vocabulary)

for word in range(len(unique_vocabulary)):
    for document in range(len(binary_bag)):
        if binary_bag[document][word] == 1:
            num_docs_including_word[word] += 1
            if document in pos_indices:
                num_pos_docs_including_word[word] += 1
            if document in neg_indices:
                num_neg_docs_including_word[word] += 1


print(num_docs_including_word)
print(num_pos_docs_including_word)
print(num_neg_docs_including_word)

[21, 90, 2, 21, 1, 93, 84, 6, 9, 95, 2, 41, 14, 26, 17, 19, 91, 25, 15, 29, 34, 64, 10, 8, 63, 6, 40, 25, 1, 79, 7, 6, 22, 12, 21, 51, 9, 13, 36, 7, 33, 26, 18, 2, 1, 4, 4, 4, 18, 11, 12, 17, 54, 3, 49, 7, 24, 4, 2, 36, 7, 2, 1, 1, 1, 40, 50, 5, 9, 1, 5, 1, 52, 16, 4, 32, 5, 8, 35, 2, 4, 7, 14, 16, 4, 26, 1, 5, 1, 5, 6, 2, 50, 13, 20, 15, 7, 11, 9, 8, 9, 14, 3, 2, 29, 1, 6, 4, 1, 10, 16, 7, 2, 2, 5, 1, 1, 9, 2, 1, 18, 1, 2, 11, 1, 6, 15, 1, 6, 20, 11, 2, 8, 7, 1, 1, 2, 3, 4, 1, 1, 1, 3, 12, 7, 10, 40, 1, 8, 4, 5, 3, 3, 1, 1, 1, 11, 5, 11, 3, 3, 3, 15, 1, 10, 2, 7, 1, 3, 5, 1, 30, 6, 2, 17, 32, 26, 5, 2, 6, 12, 4, 4, 1, 3, 7, 10, 2, 11, 18, 3, 11, 2, 5, 9, 2, 1, 6, 9, 4, 7, 12, 1, 1, 4, 2, 2, 2, 1, 20, 8, 5, 4, 2, 11, 5, 10, 10, 1, 10, 1, 4, 20, 8, 2, 19, 7, 10, 7, 8, 1, 18, 3, 1, 9, 1, 1, 1, 12, 7, 6, 1, 15, 3, 1, 6, 2, 1, 3, 2, 2, 1, 1, 16, 1, 3, 1, 5, 1, 1, 2, 3, 1, 3, 7, 2, 3, 1, 3, 1, 3, 14, 7, 1, 2, 1, 2, 1, 1, 27, 3, 2, 3, 2, 2, 4, 1, 4, 1, 6, 2, 2, 4, 6, 1, 1, 3, 1, 3, 1, 1, 1, 

In [162]:
print(binary_bag[2])

[0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [163]:
prob_word_given_pos_class = []
prob_word_given_neg_class = []

count_pos_class = math.fsum(num_pos_docs_including_word)
count_neg_class = math.fsum(num_neg_docs_including_word)


for x in range(len(unique_vocabulary)):
    prob_word_given_pos_class.append(float(num_pos_docs_including_word[x] + 1) / float(count_pos_class + len(unique_vocabulary)))
    prob_word_given_neg_class.append(float(num_neg_docs_including_word[x] + 1) / float(count_neg_class + len(unique_vocabulary)))

In [164]:
test_data = test_data.reset_index(drop=True)

def naive_bayes_classifier(test_data):
    predictions = []
    # number of unique words in the vocabulary
    V = len(unique_vocabulary)
    # alpha is the smoothing factor
    alpha = 1
    
    for x in range(len(test_data)):
        pos_prob = math.log(prior_probabilities['positive'])
        neg_prob = math.log(prior_probabilities['negative'])
        document_words = test_data['content'][x].split()

        for word in document_words:
            word = word.lower().translate(translation_table)
            if word != '':
                if word in unique_vocabulary:
                    index_of_word = unique_vocabulary.index(word)
                    # Calculate word probability with Laplace smoothing
                    word_pos_prob = (num_pos_docs_including_word[index_of_word] + alpha) / (len(pos_indices) + alpha * V)
                    word_neg_prob = (num_neg_docs_including_word[index_of_word] + alpha) / (len(neg_indices) + alpha * V)
                else:
                    # Apply smoothing for words not in the vocabulary
                    word_pos_prob = alpha / (len(pos_indices) + alpha * V)
                    word_neg_prob = alpha / (len(neg_indices) + alpha * V)
                
                pos_prob += math.log(word_pos_prob)
                neg_prob += math.log(word_neg_prob)
                
                # convert to linear space so that we can compare the probabilities (add up to 1)
                pos_prob = 10**pos_prob
                neg_prob = 10**neg_prob
                
        if pos_prob >= neg_prob:
            predictions.append('positive')
        else:
            predictions.append('negative')
    return predictions

In [165]:
# predictor for a particular review
def naive_bayes_individual(review):
    # number of unique words in the vocabulary
    V = len(unique_vocabulary)
    # alpha is the smoothing factor
    alpha = 1
    
    pos_prob = math.log(prior_probabilities['positive'])
    neg_prob = math.log(prior_probabilities['negative'])
    document_words = review.split()

    for word in document_words:
        word = word.lower().translate(translation_table)
        if word != '': 
            if word in unique_vocabulary:
                index_of_word = unique_vocabulary.index(word)
                word_pos_prob = (num_pos_docs_including_word[index_of_word] + alpha) / (len(pos_indices) + V)
                word_neg_prob = (num_neg_docs_including_word[index_of_word] + alpha) / (len(neg_indices) + V)
            else:
                word_pos_prob = 1 / (len(pos_indices) + V)
                word_neg_prob = 1 / (len(neg_indices) + V)
            
            pos_prob += math.log(word_pos_prob)
            neg_prob += math.log(word_neg_prob)
            
            # convert to linear space so that we can compare the probabilities (add up to 1)
            pos_prob = 10**pos_prob
            neg_prob = 10**neg_prob

    if pos_prob >= neg_prob:
        return 'positive'
    else:
        return 'negative'

In [166]:
# extract actual labels
actual_labels = test_data['review_class'].tolist()
# get predictions
predictions = naive_bayes_classifier(test_data)

In [167]:
def calculate_metrics(predictions, actual_labels):
    # Initialize counters
    TP = FP = TN = FN = 0
    
    # Count occurrences
    for pred, actual in zip(predictions, actual_labels):
        if pred == 'positive' and actual == 'positive':
            TP += 1
        elif pred == 'positive' and actual == 'negative':
            FP += 1
        elif pred == 'negative' and actual == 'positive':
            FN += 1
        elif pred == 'negative' and actual == 'negative':
            TN += 1
    
    # Calculate metrics
    sensitivity = TP / (TP + FN) if TP + FN else 0  
    specificity = TN / (TN + FP) if TN + FP else 0
    precision = TP / (TP + FP) if TP + FP else 0
    negative_predictive_value = TN / (TN + FN) if TN + FN else 0
    accuracy = (TP + TN) / (TP + FP + FN + TN) if TP + FP + FN + TN else 0
    F_score = (2 * precision * sensitivity) / (precision + sensitivity) if precision + sensitivity else 0

    return {
        'TP': TP,
        'TN': TN,
        'FP': FP,
        'FN': FN,
        'Sensitivity (Recall)': sensitivity,
        'Specificity': specificity,
        'Precision': precision,
        'Negative Predictive Value': negative_predictive_value,
        'Accuracy': accuracy,
        'F-score': F_score
    }

metrics = calculate_metrics(predictions, actual_labels)
for metric, value in metrics.items():
    print(f"{metric}: {value}")


TP: 18
TN: 34
FP: 15
FN: 13
Sensitivity (Recall): 0.5806451612903226
Specificity: 0.6938775510204082
Precision: 0.5454545454545454
Negative Predictive Value: 0.723404255319149
Accuracy: 0.65
F-score: 0.5625
